<a href="https://colab.research.google.com/github/ESHA1101/SSOC_Pneumonia-detection-using-x-rays/blob/main/SSOCloser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
import csv
import pandas as pd
import os
import tkinter as tk
from tkinter import ttk
import glob
from pathlib import Path
import numpy as np
import cv2
import imgaug.augmenters as ia
import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, SeparableConv2D
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

In [20]:
data_paths = Path('drive/MyDrive/chest_xray')

In [27]:
train_dir = data_paths / "train"
val_dir = data_paths / "val"
test_dir = data_paths / "test"


In [28]:
normal_cases_dir = train_dir / 'NORMAL'
pneumonia_cases_dir = train_dir / 'PNEUMONIA'

In [32]:
normal_cases = normal_cases_dir.glob('*.jpeg')
pneumonia_cases = pneumonia_cases_dir.glob('*.jpeg') 

In [ ]:
train_data=[]
for i in normal_cases:
    train_data.append((i,0))          #normal cases will be labeled as 0
for i in pneumonia_cases:             #pneumonia cases will be labelled as 1
    train_data.append((i,1))
print(train_data)


In [34]:
train_dataframe=pd.DataFrame(train_data, columns=['Image','Label'])


In [35]:
normal_cases_dir = val_dir / 'NORMAL'
pneumonia_cases_dir = val_dir / 'PNEUMONIA'
normal_cases = normal_cases_dir.glob('*.jpeg')
pneumonia_cases = pneumonia_cases_dir.glob('*.jpeg') 
val_data=[]
val_label = []

In [36]:
for image in normal_cases:
    image = cv2.imread(str(image))
    image = cv2.resize(image, (224,224))
    if image.shape[2] ==1:
        image = np.dstack([image, image, image])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.astype(np.float32)/255
    label = to_categorical(0, num_classes=2)
    val_data.append(image)
    val_label.append(label)

In [37]:
for image in pneumonia_cases:
    image = cv2.imread(str(image))
    image = cv2.resize(image, (224,224))
    if image.shape[2] ==1:
        image = np.dstack([image, image, image])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.astype(np.float32)/255.
    label = to_categorical(1, num_classes=2)
    val_data.append(image)
    val_label.append(label)

In [38]:
val_data = np.array(val_data)
val_label = np.array(val_label)

In [ ]:
x_train = np.array(x_train) / 255
x_val = np.array(x_val) / 255
x_test = np.array(x_test) / 255
# resize data for deep learning 
x_train = x_train.reshape(-1, img_size, img_size, 1)
y_train = np.array(y_train)

x_val = x_val.reshape(-1, img_size, img_size, 1)
y_val = np.array(y_val)

x_test = x_test.reshape(-1, img_size, img_size, 1)
y_test = np.array(y_test)

In [39]:
seq = ia.OneOf([
    ia.Fliplr(), 
    ia.Affine(rotate=20), 
    ia.Multiply((1.2, 1.5))]) 

In [42]:
def data_gen(data, batch_size):
    # Get total number of samples in the data
    n = len(data)
    steps = n//batch_size
    
    batch_data = np.zeros((batch_size, 224, 224, 3), dtype=np.float32)
    batch_labels = np.zeros((batch_size,2), dtype=np.float32)

    indices = np.arange(n)
    
    i =0
    while True:
        np.random.shuffle(indices)
        count = 0
        next_batch = indices[(i*batch_size):(i+1)*batch_size]
        for j, idx in enumerate(next_batch):
            img_name = data.iloc[idx]['image']
            label = data.iloc[idx]['label']
            
            encoded_label = to_categorical(label, num_classes=2)

            img = cv2.imread(str(img_name))
            img = cv2.resize(img, (224,224))
 
            if image.shape[2]==1:
                iamge = np.dstack([image, image, image])
            
            original_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                        
            original_image = image.astype(np.float32)/255.
            
            batch_data[count] = original_image
            batch_labels[count] = encoded_label
            
            if label==0 and count < batch_size-2:
                augment_img1 = seq.augment_image(img)
                augment_img2 = seq.augment_image(img)
                augment_img1 = cv2.cvtColor(augment_img1, cv2.COLOR_BGR2RGB)
                augment_img2 = cv2.cvtColor(augment_img2, cv2.COLOR_BGR2RGB)
                augment_img1 = augment_img1.astype(np.float32)/255.
                augment_img2 = augment_img2.astype(np.float32)/255.

                batch_data[count+1] = augment_img1
                batch_labels[count+1] = encoded_label
                batch_data[count+2] = augment_img2
                batch_labels[count+2] = encoded_label
                count=count+2
            
            else:
                count=count+1
            
            if count==batch_size-1:
                break
            
        i+=1
        yield batch_data, batch_labels
            
        if i>=steps:
            i=0

In [67]:

def model_build():
    input_img = Input(shape=(224,224,3), name='ImageInput')
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv1_1')(input_img)
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv1_2')(x)
    x = MaxPooling2D((2,2), name='pool1')(x)
    
    x = SeparableConv2D(128, (3,3), activation='relu', padding='same', name='Conv2_1')(x)
    x = SeparableConv2D(128, (3,3), activation='relu', padding='same', name='Conv2_2')(x)
    x = MaxPooling2D((2,2), name='pool2')(x)
    
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_1')(x)
    x = BatchNormalization(name='bn1')(x)
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_2')(x)
    x = BatchNormalization(name='bn2')(x)
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_3')(x)
    x = MaxPooling2D((2,2), name='pool3')(x)
    
    x = SeparableConv2D(512, (3,3), activation='relu', padding='same', name='Conv4_1')(x)
    x = BatchNormalization(name='bn3')(x)
    x = SeparableConv2D(512, (3,3), activation='relu', padding='same', name='Conv4_2')(x)
    x = BatchNormalization(name='bn4')(x)
    x = SeparableConv2D(512, (3,3), activation='relu', padding='same', name='Conv4_3')(x)
    x = MaxPooling2D((2,2), name='pool4')(x)
    
    x = Flatten(name='flatten')(x)
    x = Dense(1024, activation='relu', name='fc1')(x)
    x = Dropout(0.7, name='dropout1')(x)
    x = Dense(512, activation='relu', name='fc2')(x)
    x = Dropout(0.5, name='dropout2')(x)
    x = Dense(2, activation='softmax', name='fc3')(x)
    
    model = Model(inputs=input_img, outputs=x)
    return model

In [ ]:
model =  model_build()
model.summary()

In [72]:

opt = Adam(lr=0.0001, decay=1e-5)
es = EarlyStopping(patience=5)
chkpt = ModelCheckpoint(filepath='best_model_todate', save_best_only=True, save_weights_only=True)
model.compile(loss='binary_crossentropy', metrics=['accuracy'],optimizer=opt)

In [ ]:
batch_size = 16
nb_epochs = 20

train_data_gen = data_gen(data=train_dataframe, batch_size=batch_size)

nb_train_steps = train_dataframe.shape[0]//batch_size


In [ ]:
normal_cases_dir = test_dir / 'NORMAL'
pneumonia_cases_dir = test_dir / 'PNEUMONIA'

normal_cases = normal_cases_dir.glob('*.jpeg')
pneumonia_cases = pneumonia_cases_dir.glob('*.jpeg')

test_data = []
test_labels = []

for img in normal_cases:
    img = cv2.imread(str(img))
    img = cv2.resize(img, (224,224))
    if img.shape[2] ==1:
        img = np.dstack([img, img, img])
    else:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)/255.
    label = to_categorical(0, num_classes=2)
    test_data.append(img)
    test_labels.append(label)
                      
for img in pneumonia_cases:
    img = cv2.imread(str(img))
    img = cv2.resize(img, (224,224))
    if img.shape[2] ==1:
        img = np.dstack([img, img, img])
    else:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)/255.
    label = to_categorical(1, num_classes=2)
    test_data.append(img)
    test_labels.append(label)
    

test_data = np.array(test_data)
test_labels = np.array(test_labels)

print("Total number of test examples: ", test_data.shape)
print("Total number of labels:", test_labels.shape)

In [ ]:
test_loss, test_score = model.evaluate(test_data, test_labels, batch_size=16)
print("Loss on test set: ", test_loss)
print("Accuracy on test set: ", test_score)

In [ ]:
preds = model.predict(test_data, batch_size=16)
preds = np.argmax(preds, axis=-1)

original_test_labels = np.argmax(test_labels, axis=-1)

print(orig_test_labels.shape)
print(preds.shape)
